In [4]:
import langchain
from langchain.llms import GPT4All
llm = GPT4All(model="./models/ggml-gpt4all-l13b-snoozy.bin", n_ctx=512, n_threads=8)
code_prompt = "Im going to ask you to perform a task, your job is to create a program in Python that accomplishes this task. Output the function enclosed in $""$. When creating the program work it out in a step by step way to be sure we have the right answer.\n\
Examples:\n\
Task: What is 5 + 8\n\
$“def solution():\n\
     Return 5 + 8”$\n\
\n\
Task: Calculate the area of a circle with radius 5\n\
$“def solution():\n\
    radius = 5\n\
    pi = 3.14159\n\
    return pi * (radius ** 2)”$\n\
\n\
Task: Print the first 20 numbers of the Fibonacci sequence\n\
$“def solution():\n\
    sequence = [0, 1]\n\
    for i in range(2, 20):\n\
        sequence.append(sequence[i-1] + sequence[i-2])\n\
    return sequence”$\n\
\n\
Task: Analyze a text file and count the occurrences of each word\n\
$“def solution(filename):\n\
    word_counts = {}\n\
    with open(filename, 'r') as file:\n\
        for line in file:\n\
            words = line.strip().split()\n\
            for word in words:\n\
                word = word.lower()\n\
                if word in word_counts:\n\
                    word_counts[word] += 1\n\
                else:\n\
                    word_counts[word] = 1\n\
    return word_counts”$\n\
\n\
Task: "

In [5]:
def parse_program(model_output):
    # Parse the program from the model output
    program = model_output.split("$")[1]
    return program

In [6]:
def create_task_program(task):
    revisor_prompt = "You are a professional software developor that is known for reviewing code and offering perfect revisions. You have been asked to review the following code and offer revisions:\n\n"
    k = 1

    #initial program draft
    program = parse_program(llm(code_prompt + task))

    for i in range(k):
        revisions = llm(revisor_prompt + program)
        program = parse_program(llm(f'A professional developer reviewed your code and had the following revisions:\n\n{revisions}\n\nCan you revise the following code with respect to the revisions:\n\n{program}\n\nOutput the function enclosed in $""$. When creating the program work it out in a step by step way to be sure we have the right answer.'))
    
    return program